In [65]:
import pandas as pd
df = pd.read_json("../task_1/data/fragments_classification.jsonl", lines=True)
df

,text,label
0,Nie uzna gola. Robben był kilka metrów w polu ...,"[[0, 8, odwrócenie]]"
1,@USER No właśnie o tym jest ten tweet 😄,[]
2,@USER @USER Widać chcą wiecej polskich mord go...,"[[23, 38, wzmocnienie]]"
3,"Idę spać bo padam na twarz, w końcu w domuuuu",[]
4,@USER Tak się poznałam z moim chłopakiem 😂 cza...,[]
...,...,...
795,@USER Wszystkiego najlepszego z okazji urodzin...,"[[5, 29, wzmocnienie]]"
796,"@USER widzę, że pewne tweety działają jak magn...",[]
797,"@USER @USER Chociaż futro ma z jenota,\nTo nie...","[[43, 52, odwrócenie], [55, 67, wzmocnienie], ..."
798,@USER Ty aby nie zacząleś ćpać przez wydumane ...,"[[13, 25, odwrócenie], [37, 54, wzmocnienie]]"


In [66]:
label_mapping = {'wzmocnienie': 1, 
                 'osłabienie': 2,
                 'odwrócenie': 3}

In [67]:
def map_labels(label_list, mapping):
    return [[start, end, mapping[label]] for start, end, label in label_list]

In [68]:
df['label'] = df['label'].apply(lambda x: map_labels(x, label_mapping))

In [69]:
df

,text,label
0,Nie uzna gola. Robben był kilka metrów w polu ...,"[[0, 8, 3]]"
1,@USER No właśnie o tym jest ten tweet 😄,[]
2,@USER @USER Widać chcą wiecej polskich mord go...,"[[23, 38, 1]]"
3,"Idę spać bo padam na twarz, w końcu w domuuuu",[]
4,@USER Tak się poznałam z moim chłopakiem 😂 cza...,[]
...,...,...
795,@USER Wszystkiego najlepszego z okazji urodzin...,"[[5, 29, 1]]"
796,"@USER widzę, że pewne tweety działają jak magn...",[]
797,"@USER @USER Chociaż futro ma z jenota,\nTo nie...","[[43, 52, 3], [55, 67, 1], [77, 83, 1], [112, ..."
798,@USER Ty aby nie zacząleś ćpać przez wydumane ...,"[[13, 25, 3], [37, 54, 1]]"


In [70]:
from transformers import AutoTokenizer

In [71]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [72]:
def get_annotated_fragments(labels: list):
    frag_dict = {}
    for label in labels:
        frag_dict[(label[0], label[1])] = label[2]
    return frag_dict

In [73]:
a = get_annotated_fragments(df['label'][798])
a

{(13, 25): 3, (37, 54): 1}

In [74]:
encoding = tokenizer(df['text'][0], return_offsets_mapping=True)
annotated_fragments = get_annotated_fragments(df['label'][0])
# Check tokens against annotated fragments
tokens_in_annotations = []
for idx, (start, end) in enumerate(encoding['offset_mapping']):
    token_text = encoding['input_ids'][idx]
    for fragment_start, fragment_end in annotated_fragments:
        # Check if token overlaps with the fragment
        if start < fragment_end and end > fragment_start:
            tokens_in_annotations.append(token_text)  # decode token id to string
            print(annotated_fragments[(fragment_start, fragment_end)])

print("Tokens in annotated fragments:", tokens_in_annotations)

3
3
3
3
3
Tokens in annotated fragments: [9152, 2063, 1057, 2480, 2532]


In [75]:
encoding

{'input_ids': [101, 9152, 2063, 1057, 2480, 2532, 2175, 2721, 1012, 26211, 2368, 2011, 18818, 11382, 26518, 6005, 2860, 1059, 14955, 2226, 10556, 6826, 24335, 1010, 1059, 22571, 22648, 21293, 18818, 17540, 2666, 4241, 4143, 10975, 4371, 4213, 3351, 2079, 2079, 16313, 3211, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'offset_mapping': [(0, 0), (0, 2), (2, 3), (4, 5), (5, 6), (6, 8), (9, 11), (11, 13), (13, 14), (15, 19), (19, 21), (22, 24), (24, 25), (26, 28), (28, 31), (32, 37), (37, 38), (39, 40), (41, 44), (44, 45), (46, 48), (48, 50), (50, 52), (52, 53), (54, 55), (55, 57), (57, 60), (60, 63), (63, 64), (65, 68), (68, 70), (71, 73), (73, 75), (76, 78), (78, 80), (80, 82), (82, 84), (85, 87), (88, 90), (90, 93), (93, 95), 

In [78]:
encoding = tokenizer(df['text'][798], return_offsets_mapping=True)
annotated_fragments = get_annotated_fragments(df['label'][798])
# Check tokens against annotated fragments
tokens_in_annotations = []
for idx, (start, end) in enumerate(encoding['offset_mapping']):
    token_text = encoding['input_ids'][idx]
    for fragment_start, fragment_end in annotated_fragments:
        # Check if token overlaps with the fragment
        if start < fragment_end and end > fragment_start:
            tokens_in_annotations.append((token_text, annotated_fragments[(fragment_start, fragment_end)]))

print("Tokens in annotated fragments:", tokens_in_annotations)

Tokens in annotated fragments: [(9152, 3), (2063, 3), (29250, 3), (16739, 3), (2229, 3), (1059, 1), (25688, 1), (19042, 1), (2063, 1), (3291, 1), (2100, 1)]


In [79]:
def process_row(row):
    encoding = tokenizer(row['text'], return_offsets_mapping=True)
    annotated_fragments = get_annotated_fragments(row['label'])
    # Check tokens against annotated fragments
    tokens_in_annotations = []
    for idx, (start, end) in enumerate(encoding['offset_mapping']):
        token_text = encoding['input_ids'][idx]
        for fragment_start, fragment_end in annotated_fragments:
            # Check if token overlaps with the fragment
            if start < fragment_end and end > fragment_start:
                tokens_in_annotations.append((token_text, annotated_fragments[(fragment_start, fragment_end)]))
    return tokens_in_annotations

In [80]:
df['tokens_in_annotations_with_labels'] = df.apply(process_row, axis=1)

In [81]:
df['tokens_in_annotations_with_labels'][798]

[(9152, 3),
 (2063, 3),
 (29250, 3),
 (16739, 3),
 (2229, 3),
 (1059, 1),
 (25688, 1),
 (19042, 1),
 (2063, 1),
 (3291, 1),
 (2100, 1)]

In [82]:
from sklearn.model_selection import train_test_split

In [83]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [84]:
import numpy as np

In [86]:
def tokenize_and_align_labels(example):
    # Tokenizacja całego tekstu
    tokenized_inputs = tokenizer(example["text"], truncation=True, return_offsets_mapping=True)
    labels = np.full(len(tokenized_inputs["input_ids"]), -100)  # Inicjalizacja wszystkich etykiet na -100
    
    # Dopasowanie etykiet do odpowiednich tokenów
    for token_id, label in example["tokens_in_annotations_with_labels"]:  # Przechodzi przez listę adnotacji (token_id, label)
        # Znajdź pozycję `token_id` w `input_ids`
        token_positions = [i for i, input_id in enumerate(tokenized_inputs["input_ids"]) if input_id == token_id]
        for pos in token_positions:
            labels[pos] = label
    
    tokenized_inputs["labels"] = labels.tolist()  # Konwertuj etykiety na listę do zapisania w `Dataset`
    tokenized_inputs.pop("offset_mapping")  # Usuwamy offset_mapping, nie jest potrzebny w dalszej części
    return tokenized_inputs

# Przekształcenie kolumny danych na odpowiednie formaty
tokenized_dataset = df.apply(tokenize_and_align_labels, axis=1)

In [88]:
tokenized_dataset[0]

{'input_ids': [101, 9152, 2063, 1057, 2480, 2532, 2175, 2721, 1012, 26211, 2368, 2011, 18818, 11382, 26518, 6005, 2860, 1059, 14955, 2226, 10556, 6826, 24335, 1010, 1059, 22571, 22648, 21293, 18818, 17540, 2666, 4241, 4143, 10975, 4371, 4213, 3351, 2079, 2079, 16313, 3211, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 3, 3, 3, 3, 3, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]}

In [93]:
import torch
from transformers import BertForTokenClassification, TrainingArguments, Trainer

# Wczytanie modelu BERT z nagłówkiem klasyfikacji tokenów
num_labels = 3
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

# Ustawienia treningowe
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trener
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
)

# Uruchomienie treningu
trainer.train()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\kajet\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>={ACCELERATE_MIN_VERSION}'`

In [91]:
# Sprawdzenie pierwszych kilku wierszy kolumny
print(df["tokens_in_annotations_with_labels"].head())
print(df["tokens_in_annotations_with_labels"].isnull().sum())  # Liczba wartości NaN

0    [(9152, 3), (2063, 3), (1057, 3), (2480, 3), (...
1                                                   []
2    [(15536, 1), (26005, 1), (3501, 1), (14955, 1)...
3                                                   []
4                                                   []
Name: tokens_in_annotations_with_labels, dtype: object
0
